In [ ]:
!pip install -U sagemaker

In [ ]:
import sagemaker

In [5]:
from sagemaker.huggingface import HuggingFace

In [6]:
role = sagemaker.get_execution_role()

In [7]:
role

'arn:aws:iam::730335253621:role/SageMakerLLMRole'

In [8]:
hyperparameters = {
    "model_id": "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T",
    "epochs": 2,
    "per_device_train_batch_size": 2,
    "lr": 2e-5
}

In [12]:
estimator = HuggingFace(
    entry_point="train.py",
    source_dir="./scripts",
    role=role,
    transformers_version="4.36",
    pytorch_version="2.1",
    py_version="py310",
    instance_type="ml.g5.xlarge",
    instance_count=1,
    output_path="s3://llm-model-artifacts-sunny/models/",
    hyperparameters=hyperparameters
)

## Run only for the training

In [20]:
estimator.fit({
    "train": "s3://llm-finetune-dataset-sunny/datasets/"
})

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2025-11-15-11-42-14-536


2025-11-15 11:42:19 Starting - Starting the training job
2025-11-15 11:42:19 Pending - Training job waiting for capacity...
2025-11-15 11:42:33 Pending - Preparing the instances for training...
2025-11-15 11:42:59 Downloading - Downloading input data...
2025-11-15 11:43:25 Downloading - Downloading the training image.....................
2025-11-15 11:47:17 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be 

In [ ]:
# estimator.latest_training_job.model_data
# estimator.model_data

In [ ]:
# Use this code to check all the accessible services inside your AWS Sagemaker

# from sagemaker import image_uris

# image_uris.retrieve(
#     framework="huggingface",
#     region="ap-south-1",   # change your region
#     version="4.37.0",
#     image_scope="inference"
# )

In [ ]:
 # instance_type="ml.g5.xlarge",

In [ ]:
# model = HuggingFaceModel(
#     model_data="s3://bucket/model.tar.gz",
#     role=role,
#     entry_point="inference.py",
#     source_dir="inference",
#     transformers_version="4.36",
#     pytorch_version="2.1",
#     py_version="py310"
# )

In [ ]:
## the code for the deployment
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

role = sagemaker.get_execution_role()

model = HuggingFaceModel(
    model_data="s3://llm-model-artifacts-sunny/models/huggingface-pytorch-training-2025-11-15-11-42-14-536/output/model.tar.gz",
    role=role,
    transformers_version="4.37.0",
    pytorch_version="2.1.0",
    py_version="py310",
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    endpoint_name="live-finetune-endpoint"
)


In [ ]:
## THIS IS JUST TO VALIDATE WHETHER MODEL WORKING OR NOT IN THE NOTEBOOK ITSELF
predictor.predict({"inputs": "Explain AWS S3"})

In [31]:
## after the deployment URL will look like this
https://runtime.sagemaker.<region>.amazonaws.com/endpoints/live-finetune-endpoint/invocations

In [4]:
import boto3, json

runtime = boto3.client("sagemaker-runtime", region_name="ap-south-1")

resp = runtime.invoke_endpoint(
    EndpointName="live-finetune-my-endpoint",
    ContentType="application/json",
    Body=json.dumps({"inputs": "hello"})
)

print(resp["Body"].read().decode())


ClientError: An error occurred (UnrecognizedClientException) when calling the InvokeEndpoint operation: The security token included in the request is invalid.